# Lecutre 3

## Recall:
TODO:

1. Define a loss function that quantifies our unhappiness with the scors across the training data
2. Come up with a way of efficiently finding the parameters that minimize the loss function (optimization)


## Loss Function
A loss function tells how good our current classifier is.

Given a dataset of examples

$$
\{(x_i, y_i)\}^N _{i=1}
$$

where $x_i$ is an image and $y_i$ is an integer label.

Loss over the dataset is a sum of loss over examples:

$$
L = \frac{1}{N} \sum _i L_i(f(x_i, W), y_i)
$$

## Multi-class SVM Loss
Let s be the predicted score coming out of the classifier $s = f(x_i, W)$

For example, if our classes were 1 for cat and 2 for dog, then $s_1$ and $s_2$ would be cat and dog scores respectively.

$y_i$ was the category of the ground truth label, which is some integer.

SO then $s_{y_i}$ would correspond to the score of the **true class** for the i'th example in the training set.

The SVM has the form:

if $$s_{y_i} \geq s_j + 1$$
then 
$$
L_i = \sum_{j \neq y_i} = 0
$$
otherwise
$$
L_i = \sum_{j\neq y_i} s_j - s_{y_i} + 1
$$

In summary:

$$
L_i = \sum_{j\neq y_i} max(0, s_j - s_{y_i} + 1)
$$

Q1: What's the min/max possible loss for SVM?

A: Our min loss is 0, if across all our classes our loss was 0.
Our max loss is infinity. Recall the hinge joint function 

Q2: At initialization W is small so all $s \approx 0$. What is the loss?

A: About $c-1$, where c is the number of classes. This is because for each class, if we loop over all incorrect classes, each pairing will have an $s$ that are about the same, so we'll get a loss of 1 as defined in our function (the safety margin).

So, we'll get $c-1$, 1's.

This is also a useful debugging tool, that you can use as a sanity check at the beginning of training to make sure your code isn't broken.

Q3: What is the sum was over all classes, including $j = y_i$?

A: The loss increases by 1. (So now $C$ instead of $C-1$)

Q4: What if we used mean instead of sum here?
$L_i = \sum_{j\neq y_i} max(0, s_j - s_{y_i} + 1)$

A: Answer wouldn't change. The number of classes is fixed ahead of time when picking the dataset. Remember, we don't actually care about the true values of the scores/loss, only that the scores are higher for the accurate label.

Q5: What if we used squared loss?
$L_i = \sum_{j\neq y_i} max(0, s_j - s_{y_i} + 1) ^2$

A: This would be different. We trade good/badness in a nonlinear way, so it would compute a different loss function.

Looks like a squared hinge loss.

Q6: When squared over not squared?

A: Things that are very bad will now be very very bad. So use squared for cases where you don't want huge misclassifications. 

## Multiclass SVM (Support Vector Machine) Loss
Example code:

$$
L_i = \sum_{j\neq y_i} max(0, s_j - s_{y_i} + 1)
$$

In [1]:
import numpy as np

def L_i_vectorized(x, y, W):
    scores = W.dot(x)
    margins = np.maximum(0, scores - scores[y] + 1)
    margins[y] = 0
    loss_i = np.sum(margins)
    return loss_i

### Suppose that we found a W such that L = 0. Is this W unique?
Recall:

$$
f(x, W) = Wx
$$

$$
L = \frac{1}{N} \sum^N_{i=1} \sum_{j\neq y_i} max(0, s_j - s_{y_i} + 1)
$$

Answer: **NO!** Not unique. For example, $2W$ is also now $L=0$. 

## Regularization
Data loss: model predictions should match training data

Regularization: Model should be "simple", so it works on test data. 

AKA, we don't want to overfit our training data.

So, we introduce a new term in our loss function:

$$
L(W) = \frac{1}{N} \sum^N_{i=1} L_i(f(x_i, W), y_i) + \lambda R(W)
$$

#### Occam's Razor:
"Among competing hypotheses, the simplest is best"
William of Ockahm, 1285-1347.

## Regularization Types

$\lambda = $ regularization strength (**hyperparameter**)

$$
L = \frac{1}{N} \sum^N_{i=1} \sum_{j\neq y_i} max(0, f(x_i; W)_j - f(x_i;W)_{y_i} + 1) + \lambda R(W)
$$

#### In common use:
**L2 regularization** $R(W) = \sum_k \sum_l W^2 _{k,l}$

L1 regularization $R(W) = \sum_k \sum_l |W _{k,l}|$

Elastic net (L1 + L2) $R(W) = \sum_k \sum_l \beta W^2 _{k,l} + |W _{k,l}|$

Max norm regularization (might see later)

Dropout (will see later)

Fancier: Batch normalization, stocastic depth

---
A nice way of thinking about L1 vs L2 is:

L1 prefers spare solutions, that drives all your entries of W to 0 except for a couple. L2 prefers to spread the W across all the values.

# Softmax Classifier
AKA: Multinominal Logistic Regression

scores = unnormalized log probabilities of the classes.
$$
P(Y=k|X=x_i) = \frac{e^{s_k} }{\sum_j e^{s_j}}
$$
where $s = f(x_i;W)$

We want to maximize the log liklihood, or (for a loss function), to minimize the negative log likelihood of the correct class:

$$
L_i = -\log P(Y=y_i | X= x_i)
$$


Q1: Sanity check: if all the s's are small and about 0, what is the loss?

A: $-\log(\frac{1}{C}) = \log(C)$


An interesting difference between softmax and SVM:

SVM will not care once a point is correctly classified. 

Softmax will always want you to push the score of the correct class to infiity. 

## Recap
- We have some dataset of (x,y)
- We have a score function
- We have a loss function (many choices: Softmax, SVM, Full loss)


# Optimization
E.g. mountain:

Slope in any direction is dot product of direction with gradient.
Direction of steepest descent is negative gradient.

Summary:
- Numerical gradient: approximate, slow, easy to write
- Analytic gradient: exact, fast, error-prone

In practice, always use analytic gradient, but check implementation with a numerical gradient. This is called a gradient check.

## Gradient Descent

In [ ]:
# Vanila gradient descent
while True:
    weights_grad = evaluate_gradient(loss_fun, data, weights)
    weights += - step_size * weights_grad # performs parameter update

step_size is an important hyperparameter - this is commonly known as the **learning rate**.

There are also different update rules which tell us how exactly to use the gradient information at every time step (e.g. Adam, SGD, etc.)


## Stochastic Gradient Descent (SGD)
Full sum for our loss function over every single image is slow and costly.

What we can instead do is approximate using a **minibatch** of examples (32, 64, 128 are common)

In [ ]:
# Vanilla Minibatch Gradient Descent

while True:
    data_batch = sample_training_data(data, 256)
    weights_grad = evaluate_gradient(loss_fun, data_batch, weights)
    weights += - step_size * weights_grad